# Test Inference Module
This notebook tests the inference module for extracting embeddings from eBird checklists.

In [5]:
import sys
sys.path.append('..')

from src.inference import EmbeddingExtractor, load_and_extract, save_embeddings, load_embeddings
from src.data.loader import load_ebird_data
from src.data.preprocessor import EBirdPreprocessor
import torch
import pandas as pd
import numpy as np

print('✓ Imports successful!')

✓ Imports successful!


## 1. Load Sample Data

In [6]:
# Load and preprocess sample data
data_path = r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\data\raw\ebd_IN-KL_smp_relSep-2025.txt'

print('Loading data...')
df = load_ebird_data(data_path, nrows=50000)

print('\nPreprocessing...')
preprocessor = EBirdPreprocessor()
processed_df = preprocessor.fit_transform(df)

print(f'\n✓ Loaded and processed {len(processed_df)} checklists')
print(f'  Feature dimensions: {processed_df.shape[1]}')

Loading data...
✓ Loaded eBird data from: ebd_IN-KL_smp_relSep-2025.txt
  Rows: 50,000
  Columns: 53
  (Limited to first 50,000 rows)

Preprocessing...
Input data: 50,000 observations
  Unique checklists: 2,982
  Unique species: 549

Applying quality filters...
  After CATEGORY='species': 46,839 observations (3,161 removed)
  After OBSERVATION TYPE filter: 32,899 observations (13,940 removed)
  After ALL SPECIES REPORTED=1: 24,912 observations (7,987 removed)

✓ Quality filters applied: 50,000 → 24,912 observations
  Removed: 25,088 (50.2%)

Filtered data:
  Unique checklists: 1,102
  Unique species: 373

Pivoting data to create species matrix...
✓ Initial matrix: 1,102 checklists × 373 species

✓ Final matrix: 1,102 checklists × 373 species

✓ Loaded and processed 1102 checklists
  Feature dimensions: 373


## 2. Test EmbeddingExtractor Class

In [7]:
# Initialize extractor with trained model
# Use the new model trained with save_model_for_inference()
model_path = r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\models\vae_model_inference_ready.pth'

print('Initializing extractor...')
extractor = EmbeddingExtractor(model_path, device='cpu')

print('\n✓ Extractor initialized')

Initializing extractor...
✓ Model loaded from C:\Users\Arnav\Documents\Python Scripts\bird embeddings\vae_model_inference_ready.pth
  Input dim: 373, Latent dim: 16, Hidden dim: 512, Device: cpu

✓ Extractor initialized


## 3. Extract Embeddings (Deterministic Mode)

In [8]:
# Extract using mean only (deterministic)
print('Extracting embeddings (deterministic mode)...')
embeddings_mean = extractor.extract_embeddings(processed_df, use_mean=True, batch_size=128)

print(f'\n✓ Extracted embeddings')
print(f'  Shape: {embeddings_mean.shape}')
print(f'  Min: {embeddings_mean.min():.4f}')
print(f'  Max: {embeddings_mean.max():.4f}')
print(f'  Mean: {embeddings_mean.mean():.4f}')
print(f'  Std: {embeddings_mean.std():.4f}')

Extracting embeddings (deterministic mode)...

✓ Extracted embeddings
  Shape: (1102, 16)
  Min: -0.1648
  Max: 0.0998
  Mean: -0.0015
  Std: 0.0266


## 4. Extract Embeddings (Stochastic Mode)

In [9]:
# Extract by sampling (stochastic)
print('Extracting embeddings (stochastic mode)...')
embeddings_sample = extractor.extract_embeddings(processed_df, use_mean=False, batch_size=128)

print(f'\n✓ Extracted embeddings')
print(f'  Shape: {embeddings_sample.shape}')
print(f'  Min: {embeddings_sample.min():.4f}')
print(f'  Max: {embeddings_sample.max():.4f}')
print(f'  Mean: {embeddings_sample.mean():.4f}')
print(f'  Std: {embeddings_sample.std():.4f}')

Extracting embeddings (stochastic mode)...

✓ Extracted embeddings
  Shape: (1102, 16)
  Min: -3.8627
  Max: 3.7742
  Mean: 0.0007
  Std: 1.0174


## 5. Compare Deterministic vs Stochastic

In [10]:
# Compare the two modes
diff = np.abs(embeddings_mean - embeddings_sample)

print('Difference between deterministic and stochastic modes:')
print(f'  Mean absolute difference: {diff.mean():.4f}')
print(f'  Max absolute difference: {diff.max():.4f}')
print(f'\nNote: Some difference is expected due to sampling in stochastic mode')

Difference between deterministic and stochastic modes:
  Mean absolute difference: 0.8103
  Max absolute difference: 3.8697

Note: Some difference is expected due to sampling in stochastic mode


## 6. Test Extract with Reconstruction

In [11]:
# Extract embeddings and reconstructions
print('Extracting embeddings with reconstructions...')
embeddings, reconstructions = extractor.extract_with_reconstruction(
    processed_df.iloc[:10],  # Just test on 10 samples
    use_mean=True
)

print(f'\n✓ Extracted embeddings and reconstructions')
print(f'  Embeddings shape: {embeddings.shape}')
print(f'  Reconstructions shape: {reconstructions.shape}')

# Compute reconstruction error
original = processed_df.iloc[:10].values
recon_error = np.mean((original - reconstructions) ** 2)
print(f'  Mean reconstruction error (MSE): {recon_error:.6f}')

Extracting embeddings with reconstructions...

✓ Extracted embeddings and reconstructions
  Embeddings shape: (10, 16)
  Reconstructions shape: (10, 373)
  Mean reconstruction error (MSE): 0.100942


## 7. Test Convenience Function

In [12]:
# Test load_and_extract convenience function
print('Testing load_and_extract convenience function...')
embeddings_quick = load_and_extract(
    model_path=model_path,
    data=processed_df.iloc[:10],
    use_mean=True,
    device='cpu'
)

print(f'\n✓ Quick extraction successful')
print(f'  Shape: {embeddings_quick.shape}')

# Verify it matches the extractor result
max_diff = np.abs(embeddings_quick - embeddings).max()
if max_diff < 1e-6:
    print(f'  ✓ Results match extractor (max diff: {max_diff:.2e})')
else:
    print(f'  ✗ Results differ (max diff: {max_diff:.2e})')

Testing load_and_extract convenience function...
✓ Model loaded from C:\Users\Arnav\Documents\Python Scripts\bird embeddings\vae_model_inference_ready.pth
  Input dim: 373, Latent dim: 16, Hidden dim: 512, Device: cpu

✓ Quick extraction successful
  Shape: (10, 16)
  ✓ Results match extractor (max diff: 0.00e+00)


## 8. Test Save/Load Embeddings (NPZ format)

In [14]:
# Save embeddings as NPZ
save_path_npz = r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.npz'
checklist_ids = pd.Series(range(len(embeddings_mean)))

print('Saving embeddings to NPZ...')
save_embeddings(
    embeddings_mean,
    save_path_npz,
    checklist_ids=checklist_ids,
    metadata={'model': 'vae_kerala', 'latent_dim': embeddings_mean.shape[1]}
)

# Load back
print('\nLoading embeddings from NPZ...')
loaded_embeddings, loaded_ids = load_embeddings(save_path_npz)

print(f'\n✓ Loaded embeddings')
print(f'  Shape: {loaded_embeddings.shape}')
print(f'  IDs shape: {loaded_ids.shape}')

# Verify
if np.allclose(loaded_embeddings, embeddings_mean):
    print('  ✓ Embeddings match original')
else:
    print('  ✗ Embeddings differ from original')

Saving embeddings to NPZ...
✓ Embeddings saved to C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.npz

Loading embeddings from NPZ...

✓ Loaded embeddings
  Shape: (1102, 16)
  IDs shape: (1102,)
  ✓ Embeddings match original


## 9. Test Save/Load Embeddings (CSV format)

In [16]:
# Save embeddings as CSV
save_path_csv = r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.csv'

print('Saving embeddings to CSV...')
save_embeddings(
    embeddings_mean[:10],  # Just save first 10 for CSV
    save_path_csv,
    checklist_ids=checklist_ids[:10]
)

# Load back
print('\nLoading embeddings from CSV...')
loaded_csv = load_embeddings(save_path_csv)

print(f'\n✓ Loaded embeddings')
print(f'  Shape: {loaded_csv.shape}')

# Verify
if np.allclose(loaded_csv, embeddings_mean[:10]):
    print('  ✓ Embeddings match original')
else:
    print('  ✗ Embeddings differ from original')

Saving embeddings to CSV...
✓ Embeddings saved to C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.csv

Loading embeddings from CSV...

✓ Loaded embeddings
  Shape: (10, 16)
  ✓ Embeddings match original


## 10. Test Different Input Types

In [17]:
# Test with different input formats
sample_data = processed_df.iloc[:5]

# 1. DataFrame input
print('Testing DataFrame input...')
emb_df = extractor.extract_embeddings(sample_data, use_mean=True)
print(f'  ✓ DataFrame: {emb_df.shape}')

# 2. Numpy array input
print('Testing Numpy array input...')
emb_np = extractor.extract_embeddings(sample_data.values, use_mean=True)
print(f'  ✓ Numpy array: {emb_np.shape}')

# 3. Tensor input
print('Testing Tensor input...')
emb_tensor = extractor.extract_embeddings(torch.FloatTensor(sample_data.values), use_mean=True)
print(f'  ✓ Tensor: {emb_tensor.shape}')

# Verify all produce same results
if np.allclose(emb_df, emb_np) and np.allclose(emb_np, emb_tensor):
    print('\n✓ All input types produce identical results')
else:
    print('\n✗ Input types produce different results')

Testing DataFrame input...
  ✓ DataFrame: (5, 16)
Testing Numpy array input...
  ✓ Numpy array: (5, 16)
Testing Tensor input...
  ✓ Tensor: (5, 16)

✓ All input types produce identical results


## 11. Cleanup Test Files

In [18]:
import os

# Remove test files
for file in [r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.npz', r'C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.csv']:
    if os.path.exists(file):
        os.remove(file)
        print(f'Removed {file}')

print('\n✓ All tests completed successfully!')

Removed C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.npz
Removed C:\Users\Arnav\Documents\Python Scripts\bird embeddings\embeddings_test.csv

✓ All tests completed successfully!
